# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [13]:
import torch
import torch.nn as nn

In [14]:
print(torch.backends.mps.is_available())

True


In [15]:
if torch.backends.mps.is_available():
    device = torch.device("mps") # Apple GPU
    print("Using MPS device")
else:
    device = torch.device("cpu") # Defaults to CPU
    print("MPS device not found, using CPU")

# Example: Move a tensor or model to the MPS device
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=5, batch_first=False, num_layers=2)
        self.left = model
        self.right = model
        self.combine = nn.Linear(10, 10)

    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            nn.GELU(),
            nn.Linear(10, 8),
            nn.GELU(),
            nn.Linear(8, 4),
            nn.GELU(),
            nn.Linear(4, 1),
            nn.Sigmoid()
        )

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel()
model2.to(device)



Using MPS device


BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 5, num_layers=2)
    (right): LSTM(3, 5, num_layers=2)
    (combine): Linear(in_features=10, out_features=10, bias=True)
  )
  (rest): Sequential(
    (0): GELU(approximate='none')
    (1): Linear(in_features=10, out_features=8, bias=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): GELU(approximate='none')
    (5): Linear(in_features=4, out_features=1, bias=True)
    (6): Sigmoid()
  )
)

In [16]:
"""
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression
"""

'\nbatch = 32\nseq_len = 64\n\nx_left  = torch.randn(seq_len, batch, 3, device=device)\nx_right = torch.randn(seq_len, batch, 3, device=device)\n\n# labels depend on problem:\ny = torch.randn(batch, 1, device=device)  # regression\n'

In [17]:
import pandas as pd
import glob
files = glob.glob("data/*.json")
people = []
for file in files:
    df = pd.read_json(file)
    drawings = []
    for drawing in df.values:
        points = drawing[0][0]["points"]
        triples = [(point["x"], point["y"], point["time"]) for point in points]
        drawings.append(triples)
    people.append(drawings)


data = people


In [18]:
import json
from pathlib import Path

out_path = Path("output/data.json")

out_path.write_text(json.dumps(data, indent=2))

485185

In [19]:
out_path = Path("output/data.json")
loaded = json.load(out_path.open())

Looking at your code, I can see several issues with the train/test split creation:

1. You're using `len(left)` but `left` is not defined - it should be `len(paired)`
2. You're overwriting `leftTrain`, `rightTrain`, and `yTrain` instead of creating separate test variables
3. The variable naming is inconsistent

Here's the fixed code for your cell:



In [20]:
import numpy as np

training: list[list[list[tuple[int, int, int]]]] = loaded
pairedDeep = [[torch.tensor(drawing, device=device) for drawing in drawings] for drawings in training]
paired: list[torch.Tensor] = []
personIDs: list[int] = []
for personID, person in enumerate(pairedDeep):
    person = [person[0][:-1]-person[0][1:] for drawing in person]
    a = [personID] * len(person)
    personIDs.extend(a)
    paired.extend([(i[:, :] - i.min(dim=0, keepdim=True).values) / (
    (i.max(dim=0, keepdim=True).values - i.min(dim=0, keepdim=True).values)) for i in person])
personIDs = np.array(personIDs)
lengths = torch.tensor([len(s) for s in paired])


def packPad(pairs: list[torch.Tensor], lengths: torch.Tensor, indeces):
    padded = nn.utils.rnn.pad_sequence([pairs[i] for i in indeces], batch_first=True)
    padded.to(device)
    lengths = lengths[indeces]
    packed = nn.utils.rnn.pack_padded_sequence(
        padded,
        lengths,
        batch_first=True,
        enforce_sorted=False
    ).float()
    packed.to(device)
    return packed


permuteLeftTrain = np.random.permutation(len(paired) * 10) % int(len(paired)*0.8 - 1)
permuteRightTrain = np.random.permutation(len(paired) * 10) % int(len(paired)*0.8 - 1)
permuteLeftTest = np.random.permutation(len(paired) * 10) % int(len(paired)*0.2 - 1) + int(len(paired)*0.8)
permuteRightTest = np.random.permutation(len(paired) * 10) % int(len(paired)*0.2 - 1) + int(len(paired)*0.8)
pLT = permuteLeftTrain
pRT = permuteRightTrain
pLTest = permuteLeftTest
pRTest = permuteRightTest

pairedNP = paired
lengthsNP = lengths
leftTrain = packPad(pairedNP, lengthsNP, pLT)
rightTrain = packPad(pairedNP, lengthsNP, pRT)
yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

leftTest = packPad(pairedNP, lengthsNP, pLTest)
rightTest = packPad(pairedNP, lengthsNP, pRTest)
yTest = (torch.tensor(personIDs[pLTest] == personIDs[pRTest])
         .reshape((-1, 1))
         .to(device, dtype=torch.float32))


In [21]:
str((model2(leftTest, rightTest).round().int()==yTest.int()).float().mean().item())

'0.4964071810245514'

In [22]:
model2.branch.left.all_weights

[[Parameter containing:
  tensor([[-0.4224,  0.2433,  0.0843],
          [ 0.4296,  0.2197, -0.3323],
          [-0.1475,  0.3272, -0.0840],
          [-0.0030, -0.3104,  0.2519],
          [-0.1105,  0.2550, -0.4460],
          [-0.0918,  0.0797,  0.3645],
          [ 0.2259, -0.3651, -0.1299],
          [-0.0019, -0.3141,  0.2737],
          [-0.3887,  0.2142,  0.1837],
          [ 0.2120,  0.0189,  0.4110],
          [ 0.3954,  0.1245, -0.1201],
          [ 0.1620, -0.0849,  0.1829],
          [-0.1150,  0.4313,  0.4022],
          [-0.0243,  0.4413, -0.3229],
          [-0.2650, -0.2677, -0.2411],
          [ 0.1241,  0.0474, -0.2808],
          [ 0.2780, -0.1942, -0.1665],
          [ 0.4411,  0.0379, -0.3708],
          [ 0.4064, -0.0844, -0.1720],
          [ 0.0770, -0.2469,  0.3562]], device='mps:0', requires_grad=True),
  Parameter containing:
  tensor([[ 0.0307,  0.0168,  0.2030, -0.1206, -0.3112],
          [ 0.4334,  0.0090,  0.2434, -0.2487, -0.0821],
          [-0.0961, 

In [23]:
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.005)

In [24]:
for epoch in range(2000):
    optimizer.zero_grad()
    epoch_loss = 0.0
    output = model2(leftTrain, rightTrain)
    loss = criterion(output, yTrain)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item() * yTrain.size(0)

    # average loss for the epoch
    epoch_loss /= len(leftTrain)



    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {(model2(leftTest, rightTest).round().int()==yTest.int()).float().mean():.4f}, {(model2(leftTrain, rightTrain).round().int()==yTrain.int()).float().mean():.4f}")

model2


Epoch   0 – loss: 273.4574 - fPC 0.4964, 0.8611
Epoch  10 – loss: 246.6835 - fPC 0.4964, 0.8611
Epoch  20 – loss: 199.0112 - fPC 0.4964, 0.8611
Epoch  30 – loss: 176.3205 - fPC 0.4964, 0.8611
Epoch  40 – loss: 168.7477 - fPC 0.4964, 0.8611
Epoch  50 – loss: 168.7341 - fPC 0.4964, 0.8611
Epoch  60 – loss: 168.6428 - fPC 0.4964, 0.8611
Epoch  70 – loss: 168.3148 - fPC 0.4964, 0.8611
Epoch  80 – loss: 168.2390 - fPC 0.4964, 0.8611
Epoch  90 – loss: 168.2387 - fPC 0.4964, 0.8611
Epoch 100 – loss: 168.2293 - fPC 0.4964, 0.8611
Epoch 110 – loss: 168.2127 - fPC 0.4964, 0.8611
Epoch 120 – loss: 168.1934 - fPC 0.4964, 0.8611
Epoch 130 – loss: 168.1640 - fPC 0.4964, 0.8611
Epoch 140 – loss: 168.1082 - fPC 0.4964, 0.8611
Epoch 150 – loss: 167.9753 - fPC 0.4964, 0.8611
Epoch 160 – loss: 167.4833 - fPC 0.4964, 0.8611
Epoch 170 – loss: 166.3579 - fPC 0.4964, 0.8611
Epoch 180 – loss: 165.1957 - fPC 0.4964, 0.8611
Epoch 190 – loss: 163.5841 - fPC 0.4964, 0.8611
Epoch 200 – loss: 161.8202 - fPC 0.4964,

BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 5, num_layers=2)
    (right): LSTM(3, 5, num_layers=2)
    (combine): Linear(in_features=10, out_features=10, bias=True)
  )
  (rest): Sequential(
    (0): GELU(approximate='none')
    (1): Linear(in_features=10, out_features=8, bias=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): GELU(approximate='none')
    (5): Linear(in_features=4, out_features=1, bias=True)
    (6): Sigmoid()
  )
)

In [25]:
import torch

def confusion_rates(yhat: torch.Tensor, y: torch.Tensor):
    tp = (yhat == 1) & (y == 1)
    tn = (yhat == 0) & (y == 0)
    fp = (yhat == 1) & (y == 0)
    fn = (yhat == 0) & (y == 1)

    TP = int(tp.sum().item())
    TN = int(tn.sum().item())
    FP = int(fp.sum().item())
    FN = int(fn.sum().item())

    eps = 1e-12
    TPR = TP / (TP + FN + eps)   # recall / sensitivity
    TNR = TN / (TN + FP + eps)   # specificity
    FPR = FP / (FP + TN + eps)   # 1 - specificity
    FNR = FN / (FN + TP + eps)   # 1 - recall

    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN,
            "TPR": TPR, "TNR": TNR, "FPR": FPR, "FNR": FNR}

def metrics():
    yhat = model2(leftTest, rightTest).round().int()
    y = yTest
    return confusion_rates(yhat, y)

res = metrics()

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}")
print(f"False Positive Rate         : {res['FPR']:.4f}")
print(f"False Negative Rate         : {res['FNR']:.4f}")


True Positive Rate (Recall) : 1.0000
True Negative Rate (Spec.)  : 1.0000
False Positive Rate         : 0.0000
False Negative Rate         : 0.0000
